In [1]:
import pandas as pd
import numpy as np

In [2]:
from collections import Counter 

In [3]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [5]:
import ast

In [6]:
bills_uncat = pd.read_csv('../../cc_legislation/data/bills_with_cluster_count.csv')

In [7]:
prime_sponsors = pd.read_csv('../../cc_legislation/data/prime_sponsors_community.csv')

In [8]:
ps = prime_sponsors[['EventItemMatterFile', 'MatterSponsorName']]

In [9]:
ps.columns = ['EventItemMatterFile', 'primary_sponsor']

In [10]:
bills_uncat = bills_uncat.merge(ps, on='EventItemMatterFile', how='left')

In [11]:
bills_uncat['total_sponsors'] = bills_uncat['cluster_0'] + bills_uncat['cluster_1'] + bills_uncat['cluster_2'] + bills_uncat['cluster_3'] + bills_uncat['cluster_4']

In [12]:
bills_uncat['percent_cluster_3'] = round(bills_uncat['cluster_3'] / bills_uncat['total_sponsors'] * 100, 2)

In [115]:
# display max col width
pd.set_option('display.max_colwidth', None)

In [13]:
bills_uncat[bills_uncat['total_sponsors'] > 1][['sponsors','total_sponsors','EventItemMatterStatus','EventItemMatterFile','EventItemMatterName','percent_cluster_3']].sort_values('percent_cluster_3', ascending=False).head(10)

,sponsors,total_sponsors,EventItemMatterStatus,EventItemMatterFile,EventItemMatterName,percent_cluster_3
1047,"['Tiffany Cabán', 'Shahana K. Hanif', 'Lincoln...",5,Filed (End of Session),Int 1144-2023,Requiring building owners to provide informati...,100.00
1268,"['Lincoln Restler', 'Jennifer Gutiérrez']",2,Filed (End of Session),Int 1276-2023,Requiring taxis and for-hire vehicles to displ...,100.00
265,"['Lincoln Restler', 'Shahana K. Hanif', 'Tiffa...",3,Filed (End of Session),Int 0189-2022,Developing a college admissions counseling pro...,100.00
778,"['Chi A. Ossé', 'Shahana K. Hanif', 'Tiffany C...",3,Filed (End of Session),Int 1053-2023,Requiring low-wage workers to enter into coven...,100.00
707,"['Lincoln Restler', 'Shahana K. Hanif', 'Publi...",2,Filed (End of Session),Int 0736-2022,Requiring a study and mitigation of the impact...,100.00
872,"['Selvena N. Brooks-Powers', 'Kristin Richards...",5,Filed (End of Session),Int 0900-2023,Enhancing penalties for sidewalk parking and i...,80.00
917,"['Kristin Richardson Jordan', 'Chi A. Ossé', '...",10,Filed (End of Session),Int 0837-2022,Wrongful discharge from employment.,80.00
285,"['Alexa Avilés', 'Chi A. Ossé', 'Kristin Richa...",4,Withdrawn,Int 0144-2022,Creating an elected civilian review board and ...,75.00
695,"['Sandy Nurse', 'Chi A. Ossé', 'Crystal Hudson...",8,Filed (End of Session),Int 1054-2023,Requiring the licensing of last-mile distribut...,75.00
995,"['Kristin Richardson Jordan', 'Crystal Hudson'...",7,Filed (End of Session),Int 0925-2023,"NYC plumbing code, and the NYC building code, ...",71.43


In [14]:
bills_uncat['percent_cluster_3'].median()

21.43

In [15]:
three = bills_uncat[bills_uncat['primary_sponsor_cluster'] == 3]

In [16]:
one = bills_uncat[bills_uncat['primary_sponsor_cluster'] == 1]
two = bills_uncat[bills_uncat['primary_sponsor_cluster'] == 2]
four = bills_uncat[bills_uncat['primary_sponsor_cluster'] == 4]

In [104]:
three.columns

Index(['MatterSponsorMatterId', 'sponsors', 'cluster_0', 'cluster_1',
       'cluster_2', 'cluster_3', 'cluster_4', 'primary_sponsor_cluster',
       'EventItemMatterStatus', 'EventItemMatterFile', 'EventItemMatterName',
       'total_sponsors', 'percent_cluster_3', 'prepared_text', 'joined_text'],
      dtype='object')

In [17]:
three[three['EventItemMatterStatus'] == 'Enacted']['primary_sponsor'].value_counts()

primary_sponsor
tiffany cabán         7
jennifer gutiérrez    6
shahana k. hanif      5
sandy nurse           4
alexa avilés          4
chi a. ossé           3
lincoln restler       2
Name: count, dtype: int64

In [111]:
three[three['EventItemMatterStatus'] == 'Filed (End of Session)']['primary_sponsor'].value_counts()

primary_sponsor
lincoln restler              22
sandy nurse                  19
shahana k. hanif             17
alexa avilés                 17
tiffany cabán                15
chi a. ossé                  15
jennifer gutiérrez           14
kristin richardson jordan     6
Name: count, dtype: int64

In [122]:
two[two['EventItemMatterStatus'] == 'Enacted'][['primary_sponsor','total_sponsors','EventItemMatterFile','EventItemMatterName','percent_cluster_3']].sort_values('total_sponsors',ascending=False).head(25)

,primary_sponsor,total_sponsors,EventItemMatterFile,EventItemMatterName,percent_cluster_3
981,lynn c. schulman,43,Int 0918-2023,Requiring the department of health and mental hygiene to develop and implement a citywide diabetes incidence and impact reduction plan.,16.28
144,justin l. brannan,43,Int 0102-2022,Requiring the department of environmental protection to include information about the location of green roofs in an online map published by such department.,18.60
916,julie menin,41,Int 0844-2022,Establishing an office of healthcare accountability.,19.51
371,james f. gennaro,41,Int 0470-2022,Phasing out the use of fuel oil grade no. 4.,19.51
64,diana i. ayala,41,Int 0006-2022,Remediation of lead-based paint in dwelling units in multiple dwellings.,19.51
174,julie menin,41,Int 0116-2022,Creating a one-stop shop small business portal.,9.76
519,marjorie velázquez,40,Int 0559-2022,"Restricting the provision of eating utensils, condiment packets, napkins and extra eating containers, and clarifying the definition of third-party courier service.",20.00
614,julie menin,40,Int 0485-2022,Electronic child care directory.,20.00
613,julie menin,39,Int 0487-2022,Establishing a child care subsidy information portal.,17.95
621,julie menin,38,Int 0486-2022,Establishing a child care advisory board.,15.79


tf_idf 

In [18]:
add_stop = ['department','requiring','establishing','city', 'report','program', 'information','office','develop','new',
            'citywide', 'related', 'certain','regarding','use','dept','new','york','services', 'reporting','public',
            'plan','study','board','prohibiting']

In [19]:
def preprocess_data(documents):
    stop_words = set(stopwords.words('english'))
    stop_words.update(add_stop)

    tokenized_docs = [word_tokenize(doc.lower()) for doc in documents]
    processed_docs = [[word for word in doc if word not in stop_words and word.isalpha()] for doc in tokenized_docs]
    return processed_docs

In [20]:
def join_tokens(token_list):
    return ' '.join(token_list)

In [21]:
three['prepared_text'] = three['EventItemMatterName'].apply(lambda x: preprocess_data([x])[0])

C:\Users\samtg\AppData\Local\Temp\ipykernel_3980\1498695498.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  three['prepared_text'] = three['EventItemMatterName'].apply(lambda x: preprocess_data([x])[0])


In [22]:
all_tokens = [token for sublist in three['prepared_text'] for token in sublist]
word_freq = Counter(all_tokens)

most_common_words = word_freq.most_common(80)  
common_words_list = [word for word, freq in most_common_words]
common_words_string = ', '.join(f"'{word}'" for word in common_words_list)
print(common_words_string)

'housing', 'waste', 'outreach', 'emergency', 'residential', 'health', 'parking', 'rental', 'assistance', 'provide', 'individuals', 'care', 'vehicles', 'social', 'parks', 'sanitation', 'penalties', 'workers', 'food', 'powered', 'mobility', 'air', 'electric', 'violence', 'police', 'providing', 'devices', 'education', 'commercial', 'designated', 'languages', 'reports', 'complaints', 'library', 'commissioner', 'community', 'within', 'mental', 'hygiene', 'service', 'building', 'adults', 'administrative', 'quality', 'used', 'unlawful', 'tax', 'opioid', 'domestic', 'create', 'response', 'right', 'establishment', 'expiration', 'repeal', 'permits', 'civil', 'immigration', 'homeless', 'requirements', 'transportation', 'task', 'force', 'free', 'truck', 'units', 'solar', 'power', 'systems', 'safety', 'resources', 'administration', 'data', 'residents', 'property', 'recreation', 'agriculture', 'mitigation', 'zones', 'stations'


In [23]:
bills_uncat['prepared_text'] = bills_uncat['EventItemMatterName'].apply(lambda x: preprocess_data([x])[0])

In [24]:
bills_uncat['joined_text'] = bills_uncat['prepared_text'].apply(join_tokens)

In [25]:
print(bills_uncat['joined_text'].iloc[0])  # Check the first element
print(type(bills_uncat['joined_text'].iloc[0]))  # Check the type


landlords test apartment air quality offering lease signature
<class 'str'>


In [26]:
bills_uncat['joined_text'] = bills_uncat['joined_text'].apply(lambda x: x.split())

In [27]:
bill_primarysp_grouped = bills_uncat.groupby('primary_sponsor_cluster')['joined_text'].apply(lambda x: ' '.join([' '.join(sublist) for sublist in x])).reset_index()

In [28]:
all_tokens = [token for sublist in bills_uncat['joined_text'] for token in sublist]
word_freq = Counter(all_tokens)

most_common_words = word_freq.most_common(80)  
common_words_list = [word for word, freq in most_common_words]
common_words_string = ', '.join(f"'{word}'" for word in common_words_list)
print(common_words_string)

'health', 'housing', 'mental', 'service', 'provide', 'education', 'emergency', 'parking', 'police', 'creation', 'food', 'force', 'creating', 'task', 'schools', 'vehicles', 'school', 'community', 'providing', 'penalties', 'requirements', 'buildings', 'business', 'care', 'establishment', 'hygiene', 'facilities', 'commercial', 'property', 'individuals', 'repeal', 'violations', 'assistance', 'parks', 'correction', 'employees', 'access', 'development', 'outreach', 'reports', 'fire', 'increasing', 'safety', 'residential', 'code', 'programs', 'homeless', 'within', 'complaints', 'children', 'persons', 'training', 'street', 'vehicle', 'small', 'provision', 'workers', 'electric', 'tax', 'data', 'civil', 'building', 'upon', 'shelter', 'pilot', 'devices', 'system', 'transportation', 'construction', 'rental', 'powered', 'water', 'online', 'conduct', 'waste', 'centers', 'delivery', 'child', 'section', 'expiration'


In [29]:
tfidf = TfidfVectorizer()

In [30]:
bill_matrix = tfidf.fit_transform(bill_primarysp_grouped['joined_text'])
feature_names = tfidf.get_feature_names_out()
bill_tfidf_df = pd.DataFrame(bill_matrix.toarray(), columns=feature_names, index=bill_primarysp_grouped['primary_sponsor_cluster'])

In [34]:
top_words_by_cluster = {}
for cluster in bill_tfidf_df.index:
    # Sort the TF-IDF scores for the cluster in descending order
    sorted_indices = bill_tfidf_df.loc[cluster].sort_values(ascending=False).head(11)
    top_words_by_cluster[cluster] = sorted_indices.index.tolist()

# Step 5: Display the top 10 words for each cluster
for cluster, words in top_words_by_cluster.items():
    print(f"Cluster {cluster}: {', '.join(words)}")

Cluster 0.0: housing, health, establishment, creating, education, provide, buildings, mental, campaign, correction, package
Cluster 1.0: parking, service, vehicles, street, veteran, notification, electric, fire, requests, force, violations
Cluster 2.0: health, mental, food, business, education, hygiene, children, service, provide, housing, improvement
Cluster 3.0: housing, outreach, library, emergency, waste, residential, health, biometric, assistance, rental, provide
Cluster 4.0: education, police, schools, health, inspections, buildings, indoor, mental, vehicles, housing, powered


In [35]:
top_words_by_cluster

{0.0: ['housing',
  'health',
  'establishment',
  'creating',
  'education',
  'provide',
  'buildings',
  'mental',
  'campaign',
  'correction',
  'package'],
 1.0: ['parking',
  'service',
  'vehicles',
  'street',
  'veteran',
  'notification',
  'electric',
  'fire',
  'requests',
  'force',
  'violations'],
 2.0: ['health',
  'mental',
  'food',
  'business',
  'education',
  'hygiene',
  'children',
  'service',
  'provide',
  'housing',
  'improvement'],
 3.0: ['housing',
  'outreach',
  'library',
  'emergency',
  'waste',
  'residential',
  'health',
  'biometric',
  'assistance',
  'rental',
  'provide'],
 4.0: ['education',
  'police',
  'schools',
  'health',
  'inspections',
  'buildings',
  'indoor',
  'mental',
  'vehicles',
  'housing',
  'powered']}

Attempt to export words

In [ ]:

max_words = max(len(words) for words in top_words_by_cluster.values())  # Determine the max number of words in any cluster
df = pd.DataFrame.from_dict(
    {cluster: words + [None] * (max_words - len(words)) for cluster, words in top_words_by_cluster.items()},
    orient='index'
)

# Rename columns to reflect word positions
df.columns = [f'Word {i+1}' for i in range(max_words)]
df.index.name = 'Cluster'


In [39]:
df.reset_index().to_csv('../../chapter_two/data/top_words_by_cluster.csv', index=False)